# Applied Machine Learning Homework 4
Due 12/15/21 11:59PM EST

### Q1: Natural Language Processing

We will train a supervised training model to predict if a tweet has a positive or negative sentiment.

#### Dataset loading & dev/test splits

1.1) Load the twitter dataset from NLTK library

In [1]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples 

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/arunitmaity/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


1.2) Load the positive & negative tweets

In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

1.3) Create a development & test split (80/20 ratio):

In [3]:
#code here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
column_names=['Tweets','Sentiment']
#df = pd.DataFrame(columns = column_names)
dic={}
for x in range(len(positive_tweets)):
    dic[x]=[positive_tweets[x],'positive']
a=len(dic)
for x in range(len(negative_tweets)):
    dic[x+a]=[negative_tweets[x],'negative']
df=pd.DataFrame.from_dict(dic, orient='index',columns=column_names)
dev_test, text_test, dev_y, test_y = train_test_split(df['Tweets'],df['Sentiment'],test_size=0.2,stratify=df['Sentiment'])

#### Data preprocessing

We will do some data preprocessing before we tokenize the data. We will remove `#` symbol, hyperlinks, stop words & punctuations from the data. You can use the `re` package in python to find and replace these strings. 

1.4) Replace the `#` symbol with '' in every tweet

In [4]:
#code here
import regex as re
dev_test.replace('#','',regex=True,inplace=True)
text_test.replace('#','',regex=True,inplace=True)

1.5) Replace hyperlinks with '' in every tweet

In [5]:
#code here
dev_test.replace('http[^\s]+','',regex=True,inplace=True)
text_test.replace('http[^\s]+','',regex=True,inplace=True)

1.6) Remove all stop words

In [6]:
#code here
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
dev_test=dev_test.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
text_test=text_test.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arunitmaity/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1.7) Remove all punctuations

In [7]:
#code here
dev_test=dev_test.replace('[^\w\s]','')
text_test=text_test.replace('[^\w\s]','')

1.8) Apply stemming on the development & test datasets using Porter algorithm

In [8]:
#code here
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
porter=PorterStemmer()
def stemm(data):
    for i in data.index:
        tokenized_words=word_tokenize(data[i])
        tokenized_sentence = []
        for word in tokenized_words:
            tokenized_sentence.append(porter.stem(word))
        tokenized_sentence = " ".join(tokenized_sentence)
        data[i]=tokenized_sentence
    return data
dev_test=stemm(dev_test)
text_test=stemm(text_test)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arunitmaity/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Model training

1.9) Create bag of words features for each tweet in the development dataset

In [9]:
#code here
from sklearn.feature_extraction.text import CountVectorizer
vector=CountVectorizer()
dev_X=vector.fit_transform(dev_test)
text_x= vector.transform(text_test)

1.10) Train a supervised learning model of choice on the development dataset

In [10]:
#code here
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(max_iter=1000).fit(dev_X,dev_y)

1.11) Create TF-IDF features for each tweet in the development dataset

In [11]:
#code here
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
dev_X1=vectorizer.fit_transform(dev_test)
text_x1= vectorizer.transform(text_test)

1.12) Train the same supervised learning algorithm on the development dataset with TF-IDF features

In [12]:
#code here
lr1 = LogisticRegressionCV(max_iter=1000).fit(dev_X1,dev_y)

1.13) Compare the performance of the two models on the test dataset

In [13]:
#code here
print('Performance of BOG features: ', lr.score(text_x,test_y))
print('Performance of TF-IDF features: ', lr1.score(text_x1,test_y))

Performance of BOG features:  0.761
Performance of TF-IDF features:  0.7665
